In [1]:

import pandas as pd

# Load the dataset
data = pd.read_csv('order_with_trip_info.csv')

/tmp/ipykernel_48487/2257615915.py:4: DtypeWarning: Columns (7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('order_with_trip_info.csv')


In [15]:
data['order_id'].nunique()

757

In [6]:
# Split 'trip_origin' into separate latitude and longitude columns
data[['origin_lat', 'origin_lng']] = data['trip_origin'].str.strip('"').str.split(',', expand=True).astype(float)

In [7]:
# Check for missing values in the relevant columns
missing_data = data[['lat', 'lng', 'origin_lat', 'origin_lng']].isnull().sum()
print("Missing values in columns:\n", missing_data)

# Drop rows with missing values in the relevant columns
data = data.dropna(subset=['lat', 'lng', 'origin_lat', 'origin_lng'])

Missing values in columns:
 lat           1531282
lng           1531282
origin_lat    1531282
origin_lng    1531282
dtype: int64


In [11]:

# Filter the data for the specific order_id
order_id = 392001
specific_order_data = data[data['order_id'] == order_id]

if specific_order_data.empty:
    print(f"No data found for order_id {order_id}")
else:
    # Function to calculate the distance (optional, for display)
    def calculate_distance(row):
        start = (row['lat'], row['lng'])
        end = (row['origin_lat'], row['origin_lng'])
        return geodesic(start, end).kilometers

In [12]:

# Calculate distance for the specific order (optional)
specific_order_data['distance_to_origin'] = specific_order_data.apply(calculate_distance, axis=1)
print(specific_order_data[['lat', 'lng', 'origin_lat', 'origin_lng', 'distance_to_origin']])

        lat       lng  origin_lat  origin_lng  distance_to_origin
0  6.601042  3.276634    6.601042    3.276634                 0.0
1  6.565088  3.384441    6.565088    3.384441                 0.0
2  6.663648  3.308206    6.663648    3.308206                 0.0
3  6.430817  3.434155    6.430817    3.434155                 0.0
4  6.499156  3.358517    6.499156    3.358517                 0.0
5  6.535043  3.331624    6.535043    3.331624                 0.0
6  6.446071  3.478035    6.446071    3.478035                 0.0
7  6.594333  3.300784    6.594333    3.300784                 0.0
8  6.539903  3.362133    6.539903    3.362133                 0.0
9  6.510830  3.334219    6.510830    3.334219                 0.0


/tmp/ipykernel_48487/1726109242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_order_data['distance_to_origin'] = specific_order_data.apply(calculate_distance, axis=1)


In [13]:

# Create a map centered around the specific order's origin
map_center = [specific_order_data['lat'].mean(), specific_order_data['lng'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Function to add a route to the map
def add_route(row):
    start = [row['lat'], row['lng']]
    end = [row['origin_lat'], row['origin_lng']]
    folium.Marker(location=start, popup='Start').add_to(mymap)
    folium.Marker(location=end, popup='Origin').add_to(mymap)
    folium.PolyLine([start, end], color='blue').add_to(mymap)

In [14]:

# Apply the function to the specific order row
specific_order_data.apply(add_route, axis=1)

# Save the map to an HTML file
file_path = 'specific_order_route_map.html'
mymap.save(file_path)

In [8]:
from geopy.distance import geodesic

# Function to calculate the distance
def calculate_distance(row):
    start = (row['lat'], row['lng'])
    end = (row['origin_lat'], row['origin_lng'])
    return geodesic(start, end).kilometers

# Apply the function to calculate distances
data['distance_to_origin'] = data.apply(calculate_distance, axis=1)
print(data[['lat', 'lng', 'origin_lat', 'origin_lng', 'distance_to_origin']])

            lat       lng  origin_lat  origin_lng  distance_to_origin
0      6.601042  3.276634    6.601042    3.276634                 0.0
1      6.565088  3.384441    6.565088    3.384441                 0.0
2      6.663648  3.308206    6.663648    3.308206                 0.0
3      6.430817  3.434155    6.430817    3.434155                 0.0
4      6.499156  3.358517    6.499156    3.358517                 0.0
...         ...       ...         ...         ...                 ...
26453  6.442073  3.478872    6.442073    3.478872                 0.0
26454  6.446199  3.549756    6.446199    3.549756                 0.0
26455  6.482082  3.382865    6.482082    3.382865                 0.0
26456  6.449419  3.401645    6.449419    3.401645                 0.0
26457  6.443374  3.552312    6.443374    3.552312                 0.0

[26458 rows x 5 columns]


/tmp/ipykernel_48487/2634718197.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['distance_to_origin'] = data.apply(calculate_distance, axis=1)


In [9]:
import folium

# Create a map centered around the first point
map_center = [data['lat'].mean(), data['lng'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Function to add a route to the map
def add_route(row):
    start = [row['lat'], row['lng']]
    end = [row['origin_lat'], row['origin_lng']]
    folium.Marker(location=start, popup='Start').add_to(mymap)
    folium.Marker(location=end, popup='Origin').add_to(mymap)
    folium.PolyLine([start, end], color='blue').add_to(mymap)

# Apply the function to each row in the DataFrame
data.apply(add_route, axis=1)

# Save the map to an HTML file
mymap.save('routes_map.html')